# Notebook E-tivity 1 CE4021

Student name: Pierce Currid

Student ID: 23260092

<hr style="border:2px solid gray"> </hr>

## Imports

In [1]:
#None

If you believe required imports are missing, please contact your moderator.

<hr style="border:2px solid gray"> </hr>

## Task

You may add as many cells as you require to complete this task. Refer to the Rubric for E-tivity 1 to see how you will be assessed.


Proposed Steps to differentiate the polynomial:
1. validate the input. I think the main thing here is negative integers. Hmmm
2. Divide and conquer, for each term we want to:
    1. derive the term whereby X**n changes to nX**n-1
    2. add the derived term to an 'results' data structure
3. return the results data structure
4. Display it in some meaningful manner.
5. Test

Proposed steps to then evaluate a polynomial based on a specified value for its variable:
1. validate the input. I think the main thing here first checking its an integer, and a positive one at that. Hmmm
2. Calculate the result based on the value
3. Show the output as a scalar.
4. Test

Data Structures
I like the idea of Dictionaries, as they can be changed and are ordered....
in theory we could do an in-place update of the inputted values with the derived values, which would be a big memory saving.
but....Dictionaries can't have duplicate values, if there were dup terms in the inputted polynomial, computer would say no.
A Set gives us the same issue with input.

Revisiting the idea of an in-place update, a Tuple is immutable, so that would leave a List.
I think pragmatism might get in the way here though, as I'm a newbie python developer/time constraints, so potentially creating a new data structure for the result is the way forward. Given the result will have no need to be changeable, Tuple is probably the best bet for each Term within the Poly. However, the overall structure holding the terms, could this also then be a Tuple? A Tuple of Tuples


In [13]:
#creating the high level functions, minus the arguments (still need to figure out types)

#First iteration of the calc derivative polynomial...minus valid argument, return types, validation, etc...basically minus everything
def calc_derivative_polynomial(single_var_poly):
        """let's validate the input poly before we do anything, to confirm there's just one variable"""
        """Steps to differentiate
        1. Constants - derivative of any constant is 0(eg. 7 > 0)
        2. Power - X**n is derived by nX**n-1
        """
        output = single_var_poly
        return output



        

In [ ]:
#First iteration of the evaluate polynomial...minus valid arguments, return types, validation, etc...basically minus everything
def evaluate_polynomial(poly, v):
        """let's validate the input before we do anything"""
        scalar_output = ""
        return scalar_output
        

## Reflection

Write your reflection in below cell. With reference to the Rubric for E-tivity 1:
- Provide an accurate description of your code with advantages and disadvantages of design choices.
- Compare your approach to alternative (peer) approaches.
- Clearly describe how you have used your peers' work/input and how this has affected your own understanding / insights.

If you have not used peer input, you may state this, but your submission history in Gitlab should clearly show this is the case.